In [ ]:
#!pip install geopandas

In [2]:
import pandas as pd
import geopandas as gpd 
import numpy as np 
import matplotlib.pyplot as plt

In [3]:
!pip install --upgrade botocore==1.23.26  #Sur colab, sinon bug
!pip install --upgrade urllib3==1.22.0 #Sur colab, sinon bug
!pip install py7zr #Sur colab, sinon bug
!pip install s3fs #Sur colab, sinon bug
!git clone https://github.com/InseeFrLab/cartogether.git
%cd ./cartogether
!pip install -r requirements.txt
!pip install .
%cd ..

Defaulting to user installation because normal site-packages is not writeable
  Using cached botocore-1.23.26-py3-none-any.whl (8.5 MB)
  Attempting uninstall: botocore
    Found existing installation: botocore 1.27.59
    Uninstalling botocore-1.27.59:
      Successfully uninstalled botocore-1.27.59
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.4.1 requires botocore<1.27.60,>=1.27.59, but you have botocore 1.23.26 which is incompatible.
Defaulting to user installation because normal site-packages is not writeable
  Using cached urllib3-1.22-py2.py3-none-any.whl (132 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.13
    Uninstalling urllib3-1.26.13:
      Successfully uninstalled urllib3-1.26.13
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behav

Defaulting to user installation because normal site-packages is not writeable
Processing /home/gregoire/Python/ProjPythonENSAE2022/ProjPython/cartogether
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for cartiflette: filename=cartiflette-0.0.1-py3-none-any.whl size=19629 sha256=197b399175dade4d590c927afc94b5fc66c7ec87dd71631658f088bb1ada3597
  Stored in directory: /tmp/pip-ephem-wheel-cache-rytfhhx5/wheels/f4/b3/8f/2147229f24196ae8874440ac7bad9db2e287b08af2d61a7620
Successfully built cartiflette
  Attempting uninstall: cartiflette
    Found existing installation: cartiflette 0.0.1
    Uninstalling cartiflette-0.0.1:
      Successfully uninstalled cartiflette-0.0.1
/home/gregoire/Python/ProjPythonENSAE2022/ProjPython


In [4]:
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

In [5]:
#Base adresses -> coordonnées GPS
df_adresses= pd.read_csv("https://adresse.data.gouv.fr/data/ban/adresses/latest/csv/adresses-38.csv.gz", compression='gzip', sep=";", error_bad_lines=False)

/tmp/ipykernel_4091/2004415946.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_adresses= pd.read_csv("https://adresse.data.gouv.fr/data/ban/adresses/latest/csv/adresses-38.csv.gz", compression='gzip', sep=";", error_bad_lines=False)
/tmp/ipykernel_4091/2004415946.py:2: DtypeWarning: Columns (9,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df_adresses= pd.read_csv("https://adresse.data.gouv.fr/data/ban/adresses/latest/csv/adresses-38.csv.gz", compression='gzip', sep=";", error_bad_lines=False)


In [8]:
#Coordonnnées GPS ->coordonnées Geopandas
gpd_adresses= gpd.GeoDataFrame(df_adresses, geometry=gpd.points_from_xy(df_adresses.lon, df_adresses.lat))

In [17]:
#Base risque d'inondations niveau national
df_risque_inond=gpd.read_file("DataSets/Inondation/n_carte_inond_s.shp")

NameError: name 'df_risque' is not defined

In [18]:
df_risque_inond.head()

,typ_inond1,typ_inond2,scenario,datfinval,id_tri,urlrapport,est_synth,datappro,id_carte,id,geometry
0,03,03,None,None,FRM_TRI_MAYOTTE,N_MAYO_RISQUES_03.pdf,1,2019-12-16,CSI_030002,9794,"POLYGON ((45.14756 -12.77017, 45.14752 -12.706..."
1,02,None,01For,None,FRI_TRI_BASS,non communiqué,0,None,CSI_1981,9835,"POLYGON ((-61.76468 15.98499, -61.76433 16.042..."
2,02,None,01For,None,FRI_TRI_BASS,non communiqué,0,None,CSI_1982,9836,"POLYGON ((-61.74449 16.03433, -61.74413 16.092..."
3,02,None,02Moy,None,FRI_TRI_BASS,non communiqué,0,None,CSI_1983,9837,"POLYGON ((-61.76468 15.98499, -61.76433 16.042..."
4,02,None,02Moy,None,FRI_TRI_BASS,non communiqué,0,None,CSI_1984,9838,"POLYGON ((-61.74449 16.03433, -61.74413 16.092..."


In [13]:
#Coordonnées de l'habitation (en géometry) ->risque d'inondation
#gpd_adresses : tables geopandas qui contient les adresses du département
def adresse_to_geometry(dep,num,nom_voie,ville,code_postal):
  return(gpd_adresses[(gpd_adresses["numero"] == num) & (gpd_adresses["nom_commune"] == ville) & (gpd_adresses["nom_voie"] == nom_voie)]["geometry"])


In [14]:
#Fonction geometry ->risque 
def risque(geometry):
  risque=[]
  for k in range(len(df_risque_inond["geometry"])):
    if df_risque_inond["geometry"][k].contains(geometry):
      risque.append(df_risque_inond["scenario"][k])
  
  return(risque)

In [15]:
#Fonction finale
def adresse_to_risque(dep,num,nom_voie,ville,code_postal):
  df_risque_inond.to_csv() #conversion du fichier en csv 

  adresses_dep = pd.read_csv("https://adresse.data.gouv.fr/data/ban/adresses/latest/csv/adresses-"+dep+".csv.gz", compression='gzip', sep=";", error_bad_lines=False) #recuperation des adresses postales correspondant au département de l'utilisateur
  foyer_infos = adresses_dep[(adresses_dep["numero"] == num) & (adresses_dep["nom_commune"] == ville) & (adresses_dep["nom_voie"] == nom_voie)] 

  #Calcul du score moyen de "sécheresse" (i.e. niveau de gonflement/retrait d'argile, échelle de 1 à 3) du département considéré 
  argile_dep = np.mean(df_secheresse["niveau"])

  #Conversion des coordonnées GPS d'un point en une géométrie exploitable sur Geopandas 
  geometry = gpd.points_from_xy(foyer_infos["lon"], foyer_infos["lat"])

  #Création des dataframe avec geométrie du point à localiser 
  geo_foyer = gpd.GeoDataFrame(foyer_infos, crs="EPSG:4326", geometry=geometry)
  geo_foyer.crs
  #print(geo_foyer) #TEST 

  df_1 = df_secheresse[df_secheresse.niveau == 1]
  df_2 = df_secheresse[df_secheresse.niveau == 2]
  df_3 = df_secheresse[df_secheresse.niveau == 3]
  df_1.crs
  df_2.crs
  df_3.crs 
  df_secheresse.crs

  #print(df_secheresse.head()) #TEST

  #Cartographie des zones de sécheresse (selon le degré de gonflement/retrait des argiles) dans le département 31 
  fig,ax = plt.subplots(figsize=(10, 10)) 
  df_1.to_crs(3857).plot(ax = ax, color ='yellow', alpha = 0.6,zorder=2)
  df_2.to_crs(3857).plot(ax = ax, color ='orange', alpha = 0.6,zorder=2)
  df_3.to_crs(3857).plot(ax = ax, color ='red', alpha = 0.6,zorder=2)
  geo_foyer.to_crs(3857).plot(ax=ax, color="black",zorder=2)
  ax.set_title("Cartographie des zones de sécheresse préocupantes en Haute-Garonne")
  fig.legend()
  ctx.add_basemap(ax=ax,source=ctx.providers.OpenStreetMap.Mapnik)
  plt.show()

  #Fonction position géographique de l'habitation -> risque sécheresse précis associé
  
  
  print("Le score de sécheresse (de 1 à 5) de l'habitation est de :"+str(argile_dep*(5/3)))
  return(argile_dep*(5/3))

In [16]:
adresse_to_risque("44", 4, "Rue Fouré", "Nantes", "44000")

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().